In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [7]:
df = pd.read_csv("../data/enriched/de.zip", compression='zip')
df["skips"] = df["skips"].apply(literal_eval)

In [11]:
from nltk.stem.snowball import GermanStemmer, DutchStemmer
stemmer = GermanStemmer()

In [381]:
verben = pd.read_csv("../other/verben_list.txt", names=["verben"])

In [382]:
stemmed_verben = verben['verben'].apply(stemmer.stem)

In [421]:
verb_df = df[(df.word2+df.word1).apply(stemmer.stem).isin(stemmed_verben) | (df.word1+df.word2).apply(stemmer.stem).isin(stemmed_verben)]

In [458]:
v_df = verb_df[["frequency", "number_of_skips", "skip_variance", "skip_average", "word_1_frequency", "word_2_frequency", "symmetric", "chi", "pmi", "fisher", "ll"]]

In [459]:
ex_df = df[~(df.word1.isin(german_filter) | df.word2.isin(german_filter)) & \
  ~((df.word2+df.word1).apply(stemmer.stem).isin(stemmed_verben) | (df.word1+df.word2).apply(stemmer.stem).isin(stemmed_verben))
  ][["frequency", "number_of_skips", "skip_variance", "skip_average", "word_1_frequency", "word_2_frequency", "symmetric", "chi", "pmi", "fisher", "ll"]]

In [460]:
ex_df["label"] = pd.Series()
ex_df["label"] = 0
v_df["label"] = pd.Series()
v_df["label"] = 1

/home/emilio/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/emilio/Software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [461]:
ex_df = ex_df.sample(v_df["label"].count())

In [484]:
df_XY = v_df.append(ex_df).sample(frac=1)

In [485]:
X = df_XY.loc[:, "number_of_skips": "ll"]

In [486]:
Y = df_XY.loc[:, "label"]

In [487]:
kf = KFold(n_splits=5)

In [488]:
train_scores = []
test_scores = []
for train_index, test_index in kf.split(X.values):
    clf = RandomForestClassifier()
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, Y_train)
    train_scores.append(f1_score(clf.predict(X_train), Y_train, average="micro"))
    test_scores.append(f1_score(clf.predict(X_test), Y_test, average="micro"))
print(f"average training score: {sum(train_scores)/len(train_scores)}")
print(f"average test score: {sum(test_scores)/len(test_scores)}")

average training score: 0.9880694872436859
average test score: 0.7838827606697846
